# Caffe学习系列：Blob，Layer和Net以及对应配置文件的编写

深度网络（net）是一个组合模型，它由许多相互连接的层（layers）组合而成。Caffe就是组件深度网络的这样一种工具，它按照一定的策略，一层一层的搭建出自己的模型。它将所有的信息数据定义为blobs，从而进行便利的操作和通讯。Blob是caffe框架中一种标准的数组，一种统一的内存接口，它详细描述了信息是如何存储的，以及如何在层之间通讯的。

## 1、blob

Blobs封装了运行时的数据信息，提供了CPU和GPU的同步。从数学上来说，Blob就是一个N维数组。它是caffe中的数据操作基本单位，就像matlab中以矩阵为基本操作对象一样。只是矩阵是二维的，而Blob是N维的。N可以是2，3，4等等。对于图片数据来说，Blob可以表示为$(N \times C \times H \times W)$这样一个4D数组。其中N表示图片的数量，C表示图片的通道数，H和W分别表示图片的高度和宽度。当然，除了图片数据，Blob也可以用于非图片数据。比如传统的多层感知机，就是比较简单的全连接网络，用2D的Blob，调用innerProduct层来计算就可以了。

在模型中设定的参数，也使用Blob来表示和运算的。它的维度会根据参数的类型不同而不同。比如：在一个卷积层中，输入一张3通道图片，有96个卷积核，每个核大小为$11 \times 11$，因此这个Blob是$96 \times 3 \times 11 \times 11$。而在一个全连接层中，假设输入1024通道图片，输出1000个数据，则Blob为$1000 \times 1024$。

## 2、layer

层是网络模型的组成要素和计算的基本单位。层的类型比较多，如Data，Convolution，Pooling，ReLU，Softmax-loss，Accuracy等，一个层的定义大致如下图：

![Caffe中的Layer模型](./5-attach/layer.jpg)

从bottom进行数据的输入，计算后，通过top进行输出。图中的黄色多边形表示输入输出的数据，蓝色矩形表示层。

每一种类型的层都定义了三种关键的计算：setup，forward，backward。

setup：层的建立和初始化，以及在整个模型中的连接初始化。

forward：从bottom得到输入数据，进行计算，并将计算结果送到top，进行输出。

backward：从层的输出端top得到数据的梯度，计算当前层的梯度，并将计算结果送到bottom，向后传递。

## 3、Net

就像搭积木一样，一个Net由多个Layer组合而成。

现给出一个简单的2层神经网络的模型定义（加上mnist(DATA)层就变成三层了），先给出这个网络的拓扑。

![Net的拓扑结构举例](./5-attach/net.jpg)

第一层：name为mnist，type为Data，没有输入（bottom），只有两个输出（top），一个为data，一个为label

第二层：name为ip，type为InnerProduct，输入（bottom）数据为data，输出（top）数据为ip

第三层：name为loss，type为SoftmaxWithLoss，有两个输入（bottom），一个为ip，一个为label，有一个输出（top）为loss，没有画出来。

对应的配置文件prototxt就可以这样写：
```
name: "LogReg"
layer {
    name: "mnist"
    type: "Data"
    top: "data"
    top: "label"
    data_param {
        source: "input_leveldb"
        batch_size: 64
    }
}
layer {
    name: "ip"
    type: "InnerProduct"
    bottom: "data"
    top: "ip"
    inner_product_param {
        num_output: 2
    }
}
layer {
    name: "loss"
    type: "SoftmaxWithLoss"
    bottom: "ip"
    bottom: "label"
    top: "loss"
}
```
第一行将这个模型取名为LogReg，然后是三个layer的定义，参数都比较简单，只列出必须的参数。具体的参数定义可参见本系列的前几篇文章。